In [1]:
library(tidyverse)
library(mediation)
library(brms)
library(dplyr)
library(tidyr)
library(lme4)
library(ggplot2)

── Attaching core tidyverse packages ─────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   4.0.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.1.0     
── Conflicts ───────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors


Warning messages:
1: package ‘tidyverse’ was built under R version 4.3.3 
2: package ‘lubridate’ was built under R version 4.3.2 


Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: mvtnorm
Loading required package: sandwich
mediation: Causal Mediation Analysis
Version: 4.5.1



Warning messages:
1: package ‘mvtnorm’ was built under R version 4.3.3 
2: package ‘sandwich’ was built under R version 4.3.3 


Loading required package: Rcpp
Loading 'brms' package (version 2.22.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').

Attaching package: ‘brms’

The following object is masked from ‘package:stats’:

    ar



Warning message:
package ‘brms’ was built under R version 4.3.3 



Attaching package: ‘lme4’

The following object is masked from ‘package:brms’:

    ngrps



# Import Dataset

In [2]:
df_long <- read.csv("df_RCT.csv")

# Outcome Measures

In [8]:
outcomes <- c("fss_sum", "woods_sum", "dsq_sum", "vas_fatiguecurrent", "bdi_sum")
resp_vars <- c("mip_pre_max", 
               "smip_pre_max", "fit_pre_max", "pif_pre_max", "sindex_pre_max", "volume_pre_max",
               "id_pre_max", "slopesmip_pre_max",
               "mip_post_max", 
               "smip_post_max","fit_post_max", "pif_post_max", "sindex_post_max",
               "volume_post_max", "id_post_max", "slopesmip_post_max", "bdi_sum", "mmrc_score", "cpet_ve", "cpet_vt_peak", "cpet_o2pulse", 'cpet_vo2peak_absolute', 'cpet_vo2peak_relative', "fmd_percent")

# Regression

## Covariate List

In [9]:
library(dplyr)
library(broom)
library(ggplot2)

# --------------------------------------------------
# CONFIGURATION - Easily modify these
# --------------------------------------------------
covariates <- c('data_age', 'data_kilograms', 'data_centimeters', 'subject_female')
# Alternative examples:
# covariates <- c('data_age')
# covariates <- c('data_age', 'subject_female', 'education_years')
# covariates <- character()  # for no covariates

# --------------------------------------------------
# Filter to Visit 1 only
# --------------------------------------------------
df_v1 <- df_long %>%
  filter(time_point == 1)

# --------------------------------------------------
# Storage object for regression results
# --------------------------------------------------
results <- list()
idx <- 1

# --------------------------------------------------
# Create diagnostics PDF
# --------------------------------------------------
pdf("correlations_diagnostic_check.pdf", width = 8, height = 10)

for (y in outcomes) {
  for (x in resp_vars) {
    
    # Build formula dynamically based on covariates
    if (length(covariates) > 0) {
      formula_str <- paste(y, "~", x, "+", paste(covariates, collapse = " + "))
    } else {
      formula_str <- paste(y, "~", x)
    }
    formula <- as.formula(formula_str)
    
    # Select variables dynamically
    model_vars <- c(y, x)
    if (length(covariates) > 0) {
      model_vars <- c(model_vars, covariates)
    }
    
    # Remove missing data
    df_model <- df_v1 %>%
      dplyr::select(all_of(model_vars)) %>%
      na.omit()
    
    # Skip if insufficient data
    if (nrow(df_model) < 10) next
    
    # Fit model
    fit <- lm(formula, data = df_model)
    
    # Extract regression summary
    tidy_fit <- tidy(fit)
    r2 <- summary(fit)$adj.r.squared
    
    # Store results
    results[[idx]] <- data.frame(
      outcome = y,
      predictor = x,
      beta = tidy_fit$estimate[2],
      std_error = tidy_fit$std.error[2],
      p_value = tidy_fit$p.value[2],
      adj_r2 = r2,
      N = nrow(df_model),
      covariates_used = paste(covariates, collapse = ", ")
    )
    idx <- idx + 1
    
    # --------------------------------------------------
    # Diagnostics plots for this model
    # --------------------------------------------------
    par(mfrow=c(3,1))
    
    plot(fit, which = 1, main = paste("Residuals vs Fitted:", y, "~", x))
    plot(fit, which = 2, main = paste("Normal Q-Q:", y, "~", x))
    plot(fit, which = 3, main = paste("Scale-Location:", y, "~", x))
    
  }
}

dev.off()

# --------------------------------------------------
# Combine and save regression table
# --------------------------------------------------
results_df <- bind_rows(results)
write.csv(results_df, "visit1_regression_results.csv", row.names = FALSE)

cat("\n✅ DONE!\n")
cat("• Visit 1 regression models completed\n")
cat("• Covariates used:", paste(covariates, collapse = ", "), "\n")
cat("• Diagnostics PDF saved as: correlations_diagnostic_check.pdf\n")
cat("• Results table saved as: visit1_regression_results.csv\n")

Warning messages:
1: In plot.new() :
  Cannot open temporary file 'C:\Users\ZCooper\AppData\Local\Temp\Rtmp0cXP4B\pdf988c1d0b2ac0' for compression (reason: No such file or directory); compression has been turned off for this device
2: In model.matrix.default(mt, mf, contrasts) :
  the response appeared on the right-hand side and was dropped
3: In model.matrix.default(mt, mf, contrasts) :
  problem with term 1 in model.matrix: no columns are assigned
4: In model.matrix.default(object, data = list(bdi_sum = c(7, 7, 4,  :
  the response appeared on the right-hand side and was dropped
5: In model.matrix.default(object, data = list(bdi_sum = c(7, 7, 4,  :
  problem with term 1 in model.matrix: no columns are assigned
6: In model.matrix.default(object, data = list(bdi_sum = c(7, 7, 4,  :
  the response appeared on the right-hand side and was dropped
7: In model.matrix.default(object, data = list(bdi_sum = c(7, 7, 4,  :
  problem with term 1 in model.matrix: no columns are assigned
8: In mode


✅ DONE!
• Visit 1 regression models completed
• Covariates used: data_age, data_kilograms, data_centimeters, subject_female 
• Diagnostics PDF saved as: correlations_diagnostic_check.pdf
• Results table saved as: visit1_regression_results.csv


## OG

In [7]:
library(dplyr)
library(broom)
library(ggplot2)

# --------------------------------------------------
# Filter to Visit 1 only
# --------------------------------------------------
df_v1 <- df_long %>%
  filter(time_point == 1)

# --------------------------------------------------
# Storage object for regression results
# --------------------------------------------------
results <- list()
idx <- 1

# --------------------------------------------------
# Create diagnostics PDF
# --------------------------------------------------
pdf("correlations_diagnostic_check.pdf", width = 8, height = 10)

for (y in outcomes) {
  for (x in resp_vars) {
    
    # Build formula: outcome ~ respiratory + covariates
    formula <- as.formula(
      paste(y, "~", x, "+ data_age + subject_female")
    )
    
    # Remove missing data
    df_model <- df_v1 %>%
      dplyr::select(all_of(c(y, x, "data_age", "subject_female"))) %>%
      na.omit()
    
    # Skip if insufficient data
    if (nrow(df_model) < 10) next
    
    # Fit model
    fit <- lm(formula, data = df_model)
    
    # Extract regression summary
    tidy_fit <- tidy(fit)
    r2 <- summary(fit)$adj.r.squared
    
    # Store results
    results[[idx]] <- data.frame(
      outcome = y,
      predictor = x,
      beta = tidy_fit$estimate[2],
      std_error = tidy_fit$std.error[2],
      p_value = tidy_fit$p.value[2],
      adj_r2 = r2,
      N = nrow(df_model)
    )
    idx <- idx + 1
    
    # --------------------------------------------------
    # Diagnostics plots for this model
    # --------------------------------------------------
    par(mfrow=c(3,1))
    
    plot(fit, which = 1, main = paste("Residuals vs Fitted:", y, "~", x))
    plot(fit, which = 2, main = paste("Normal Q-Q:", y, "~", x))
    plot(fit, which = 3, main = paste("Scale-Location:", y, "~", x))
    
  }
}

dev.off()

# --------------------------------------------------
# Combine and save regression table
# --------------------------------------------------
results_df <- bind_rows(results)
write.csv(results_df, "visit1_regression_results.csv", row.names = FALSE)

cat("\n✅ DONE!\n")
cat("• Visit 1 regression models completed\n")
cat("• Diagnostics PDF saved as: correlations_diagnostic_check.pdf\n")
cat("• Results table saved as: visit1_regression_results.csv\n")


Warning message:
In plot.new() :
  Cannot open temporary file 'C:\Users\ZCooper\AppData\Local\Temp\RtmpSkdRJm\pdf303c1a968e9' for compression (reason: No such file or directory); compression has been turned off for this device



✅ DONE!
• Visit 1 regression models completed
• Diagnostics PDF saved as: correlations_diagnostic_check.pdf
• Results table saved as: visit1_regression_results.csv


# Group Comparison

In [10]:
library(dplyr)
library(broom)

# --------------------------------------------------
# Define respiratory variables
# --------------------------------------------------
resp_vars <- c("mip_pre_max", "mip_pre_max_percentpredict_1", "mip_pre_max_percentpredict_2", "mip_pre_max_percentpredict_3",
               "smip_pre_max", "fit_pre_max", "pif_pre_max", "sindex_pre_max", "volume_pre_max",
               "id_pre_max", "slopesmip_pre_max",
               "mip_post_max", "mip_post_max_percentpredict_1", "mip_post_max_percentpredict_2", "mip_post_max_percentpredict_3",
               "smip_post_max","fit_post_max", "pif_post_max", "sindex_post_max",
               "volume_post_max", "id_post_max", "slopesmip_post_max")

# --------------------------------------------------
# Create pre-post pairs
# --------------------------------------------------

df_v1 <- df_long %>%
  filter(time_point == 1)


# Extract base variable names (without pre/post)
base_vars <- unique(gsub("_(pre|post)_max.*", "", resp_vars))
base_vars <- base_vars[!base_vars %in% c("", "slopesmip")]  # Remove empty strings

# Create pairing list
pre_post_pairs <- list()

for (base_var in base_vars) {
  # Find all pre and post variables for this base variable
  pre_vars <- resp_vars[grepl(paste0("^", base_var, "_pre_max"), resp_vars)]
  post_vars <- resp_vars[grepl(paste0("^", base_var, "_post_max"), resp_vars)]
  
  # Pair them based on the suffix
  for (pre_var in pre_vars) {
    suffix <- gsub(paste0(base_var, "_pre_max"), "", pre_var)
    post_var <- paste0(base_var, "_post_max", suffix)
    
    if (post_var %in% post_vars) {
      pre_post_pairs[[length(pre_post_pairs) + 1]] <- c(pre_var, post_var)
    }
  }
}

# Special handling for slopesmip
if ("slopesmip_pre_max" %in% resp_vars & "slopesmip_post_max" %in% resp_vars) {
  pre_post_pairs[[length(pre_post_pairs) + 1]] <- c("slopesmip_pre_max", "slopesmip_post_max")
}

# --------------------------------------------------
# Function to run paired tests
# --------------------------------------------------
run_paired_tests <- function(pre_var, post_var, data) {
  # Filter to participants with both pre and post values
  paired_data <- data %>%
    dplyr::select(record_id, all_of(c(pre_var, post_var))) %>%
    na.omit()
  
  n_pairs <- nrow(paired_data)
  
  if (n_pairs < 10) {
    return(data.frame(
      pre_var = pre_var,
      post_var = post_var,
      n_pairs = n_pairs,
      pre_mean = NA,
      post_mean = NA,
      mean_diff = NA,
      t_statistic = NA,
      t_p_value = NA,
      w_statistic = NA,
      w_p_value = NA,
      stringsAsFactors = FALSE
    ))
  }
  
  # Calculate means
  pre_mean <- mean(paired_data[[pre_var]], na.rm = TRUE)
  post_mean <- mean(paired_data[[post_var]], na.rm = TRUE)
  mean_diff <- post_mean - pre_mean
  
  # Paired t-test
  t_test <- tryCatch({
    t.test(paired_data[[pre_var]], paired_data[[post_var]], paired = TRUE)
  }, error = function(e) {
    list(statistic = NA, p.value = NA)
  })
  
  # Wilcoxon signed-rank test
  w_test <- tryCatch({
    wilcox.test(paired_data[[pre_var]], paired_data[[post_var]], paired = TRUE, exact = FALSE)
  }, error = function(e) {
    list(statistic = NA, p.value = NA)
  })
  
  return(data.frame(
    pre_var = pre_var,
    post_var = post_var,
    n_pairs = n_pairs,
    pre_mean = pre_mean,
    post_mean = post_mean,
    mean_diff = mean_diff,
    t_statistic = ifelse(is.null(t_test$statistic), NA, t_test$statistic),
    t_p_value = ifelse(is.null(t_test$p.value), NA, t_test$p.value),
    w_statistic = ifelse(is.null(w_test$statistic), NA, w_test$statistic),
    w_p_value = ifelse(is.null(w_test$p.value), NA, w_test$p.value),
    stringsAsFactors = FALSE
  ))
}

# --------------------------------------------------
# Run all paired comparisons
# --------------------------------------------------
paired_results <- list()

for (i in seq_along(pre_post_pairs)) {
  pair <- pre_post_pairs[[i]]
  cat("Running comparison:", pair[1], "vs", pair[2], "\n")
  
  result <- run_paired_tests(pair[1], pair[2], df_v1)
  paired_results[[i]] <- result
}

paired_df <- bind_rows(paired_results)

# --------------------------------------------------
# Format and clean up results
# --------------------------------------------------
formatted_paired <- paired_df %>%
  mutate(
    pre_mean = round(pre_mean, 2),
    post_mean = round(post_mean, 2),
    mean_diff = round(mean_diff, 2),
    t_statistic = round(t_statistic, 3),
    t_p_value = round(t_p_value, 4),
    w_statistic = round(w_statistic, 1),
    w_p_value = round(w_p_value, 4),
    t_sig = case_when(
      t_p_value < 0.001 ~ "***",
      t_p_value < 0.01 ~ "**",
      t_p_value < 0.05 ~ "*",
      TRUE ~ ""
    ),
    w_sig = case_when(
      w_p_value < 0.001 ~ "***",
      w_p_value < 0.01 ~ "**",
      w_p_value < 0.05 ~ "*",
      TRUE ~ ""
    ),
    # Add interpretation
    direction = ifelse(mean_diff > 0, "Increase", "Decrease"),
    consistent_significance = ifelse(t_p_value < 0.05 & w_p_value < 0.05, "Yes", "No")
  ) %>%
  dplyr::select(
    pre_var, post_var, n_pairs,
    pre_mean, post_mean, mean_diff, direction,
    t_statistic, t_p_value, t_sig,
    w_statistic, w_p_value, w_sig,
    consistent_significance
  )

# --------------------------------------------------
# Print results
# --------------------------------------------------
cat("\n==============================================\n")
cat(" PRE vs POST ACTIVITY PAIRED COMPARISONS\n")
cat("==============================================\n")

for (i in 1:nrow(formatted_paired)) {
  row <- formatted_paired[i, ]
  cat(sprintf("\nComparison %d: %s vs %s\n", i, row$pre_var, row$post_var))
  cat(sprintf("N pairs: %d | Pre: %.2f | Post: %.2f | Difference: %.2f (%s)\n", 
              row$n_pairs, row$pre_mean, row$post_mean, row$mean_diff, row$direction))
  cat(sprintf("Paired t-test: t = %.3f, p = %.4f %s\n", 
              row$t_statistic, row$t_p_value, row$t_sig))
  cat(sprintf("Wilcoxon test: W = %.1f, p = %.4f %s\n", 
              row$w_statistic, row$w_p_value, row$w_sig))
  cat(sprintf("Consistent significance: %s\n", row$consistent_significance))
  cat("----------------------------------------------\n")
}

# --------------------------------------------------
# Summary of significant results
# --------------------------------------------------
sig_results <- formatted_paired %>%
  filter(consistent_significance == "Yes")

cat("\n==============================================\n")
cat(" SUMMARY: SIGNIFICANT PRE-POST DIFFERENCES\n")
cat("==============================================\n")
if (nrow(sig_results) > 0) {
  for (i in 1:nrow(sig_results)) {
    row <- sig_results[i, ]
    cat(sprintf("%d. %s: %.2f → %.2f (Δ%.2f, %s), p = %.4f\n", 
                i, gsub("_pre_max.*", "", row$pre_var),
                row$pre_mean, row$post_mean, row$mean_diff, row$direction,
                max(row$t_p_value, row$w_p_value, na.rm = TRUE)))
  }
} else {
  cat("No consistently significant pre-post differences found.\n")
}

# --------------------------------------------------
# Save results
# --------------------------------------------------
write.csv(formatted_paired, "pre_post_paired_comparisons.csv", row.names = FALSE)
cat("\nDetailed results saved to 'pre_post_paired_comparisons.csv'\n")

Running comparison: mip_pre_max vs mip_post_max 
Running comparison: mip_pre_max_percentpredict_1 vs mip_post_max_percentpredict_1 
Running comparison: mip_pre_max_percentpredict_2 vs mip_post_max_percentpredict_2 
Running comparison: mip_pre_max_percentpredict_3 vs mip_post_max_percentpredict_3 
Running comparison: smip_pre_max vs smip_post_max 
Running comparison: fit_pre_max vs fit_post_max 
Running comparison: pif_pre_max vs pif_post_max 
Running comparison: sindex_pre_max vs sindex_post_max 
Running comparison: volume_pre_max vs volume_post_max 
Running comparison: id_pre_max vs id_post_max 
Running comparison: slopesmip_pre_max vs slopesmip_post_max 

 PRE vs POST ACTIVITY PAIRED COMPARISONS

Comparison 1: mip_pre_max vs mip_post_max
N pairs: 22 | Pre: 47.68 | Post: 46.14 | Difference: -1.55 (Decrease)
Paired t-test: t = 0.467, p = 0.6453 
Wilcoxon test: W = 150.0, p = 0.4550 
Consistent significance: No
----------------------------------------------

Comparison 2: mip_pre_max_pe

# End